In [2]:
# Basic libs
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torchmetrics import F1Score
import sys; sys.path.insert(0, '..')

# Custom libs
from scripts.MHealthDataset import MHealthDataset
from scripts.Net import Net
training_data = MHealthDataset(data_dir="../data/data.csv",labels_dir="../data/labels.csv")
test_data = MHealthDataset(data_dir="../data/data.csv",labels_dir="../data/labels.csv")

train_dataloader = DataLoader(training_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)


In [3]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

model = Net(60)
print(model)

Using cuda device
Net(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=60, out_features=16, bias=True)
    (1): ReLU()
    (2): Linear(in_features=16, out_features=16, bias=True)
    (3): ReLU()
    (4): Linear(in_features=16, out_features=13, bias=True)
  )
)


# Training

In [4]:
learning_rate = 1e-3
batch_size = 64
f1_predictions = None
f1_actual = None

In [5]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        # Compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)
        
        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0
    f1_actual=None
    f1_predictions=None
    
    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            
            if f1_actual is None:
                f1_actual = y
                f1_predictions = pred.argmax(1)
            else:
                f1_actual=torch.cat((f1_actual, y), 0)
                f1_predictions=torch.cat((f1_predictions, pred.argmax(1)), 0)
            

            aa = (pred.argmax(1) == y).type(torch.float).sum().item()
            correct += aa
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")
    f1_actual = torch.tensor(f1_actual)
    f1_predictions = torch.tensor(f1_predictions)
    print(r'F1 Score: {score}'.format(score=f1(f1_predictions, f1_actual)))


In [7]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
f1 = F1Score(num_classes=13)
epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    
    test_loop(test_dataloader, model, loss_fn)
    
print("Done!")


Epoch 1
-------------------------------
loss: 4.109983  [    0/171588]
loss: 3.173311  [ 6400/171588]
loss: 3.613147  [12800/171588]
loss: 4.146756  [19200/171588]
loss: 2.737380  [25600/171588]
loss: 2.238137  [32000/171588]
loss: 1.803572  [38400/171588]
loss: 1.947339  [44800/171588]
loss: 1.185078  [51200/171588]
loss: 1.818202  [57600/171588]
loss: 0.502781  [64000/171588]
loss: 1.002612  [70400/171588]
loss: 1.496345  [76800/171588]
loss: 0.421693  [83200/171588]
loss: 1.460301  [89600/171588]
loss: 0.734526  [96000/171588]
loss: 0.449068  [102400/171588]
loss: 0.068779  [108800/171588]
loss: 0.134753  [115200/171588]
loss: 0.216599  [121600/171588]
loss: 0.525763  [128000/171588]
loss: 1.961062  [134400/171588]
loss: 2.096473  [140800/171588]
loss: 1.098246  [147200/171588]
loss: 0.342235  [153600/171588]
loss: 1.014101  [160000/171588]
loss: 0.588609  [166400/171588]
Test Error: 
 Accuracy: 35.2%, Avg loss: 2.042961 

F1 Score: 0.35215166211128235
Epoch 2
----------------------

C:\Users\Preslav\AppData\Local\Temp\ipykernel_25616\1644477939.py:43: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  f1_actual = torch.tensor(f1_actual)
C:\Users\Preslav\AppData\Local\Temp\ipykernel_25616\1644477939.py:44: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  f1_predictions = torch.tensor(f1_predictions)


loss: 2.679067  [ 6400/171588]
loss: 1.318725  [12800/171588]
loss: 5.300125  [19200/171588]
loss: 1.343539  [25600/171588]
loss: 0.558723  [32000/171588]
loss: 0.668104  [38400/171588]
loss: 0.387783  [44800/171588]
loss: 0.242640  [51200/171588]
loss: 0.801484  [57600/171588]
loss: 0.127711  [64000/171588]
loss: 0.225076  [70400/171588]
loss: 1.042193  [76800/171588]
loss: 0.124300  [83200/171588]
loss: 1.515851  [89600/171588]
loss: 0.181208  [96000/171588]
loss: 0.196577  [102400/171588]
loss: 0.010256  [108800/171588]
loss: 0.059518  [115200/171588]
loss: 0.095835  [121600/171588]
loss: 0.312435  [128000/171588]
loss: 1.484043  [134400/171588]
loss: 2.645610  [140800/171588]
loss: 0.321213  [147200/171588]
loss: 0.212555  [153600/171588]
loss: 0.755083  [160000/171588]
loss: 0.676204  [166400/171588]
Test Error: 
 Accuracy: 38.9%, Avg loss: 2.123112 

F1 Score: 0.38949692249298096
Epoch 3
-------------------------------
loss: 3.512511  [    0/171588]
loss: 2.481655  [ 6400/171588]